In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
24,270,20297127,카페오작교,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135953,6070.0,NaN,1,NaN,127.049293,37.522951,7호선 청담,398.760647,NaN
25,282,23238698,들꽃누리,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,150070,7412.0,NaN,1,NaN,126.898633,37.497079,7호선 대림(구로구청),368.888504,NaN
34,469,20017448,커피빈,코리아청담에스점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135955,6011.0,NaN,1,109,127.047216,37.525250,7호선 청담,675.120538,NaN
37,497,24327928,건대포토카페프린세스,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,143190,5073.0,NaN,1,NaN,127.068745,37.539802,7호선 건대입구,60.940672,NaN
47,586,20023467,커피빈,코리아청담성당점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135953,6074.0,NaN,2,NaN,127.050021,37.519902,7호선 청담,229.848907,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12149,135223,17254892,안다미로팬케이크,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,143915,5012.0,NaN,NaN,NaN,127.071012,37.544992,7호선 어린이대공원(세종대),355.054760,NaN
12150,135226,17252483,초코라떼,건대역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,143854,5066.0,NaN,7,NaN,127.074625,37.538335,7호선 어린이대공원(세종대),592.981894,NaN
12152,135251,17207986,카페,동네항동점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,152140,8362.0,NaN,1,NaN,126.821130,37.482852,7호선 온수(성공회대입구),567.166627,NaN
12159,135345,17243108,코끼리키즈까페,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,152815,8352.0,NaN,2,NaN,126.854261,37.486412,7호선 광명사거리,478.139096,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
24,카페오작교,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 도산대로92길 26,7호선 청담,398.760647,NaN,청담,청담역 카페오작교
25,들꽃누리,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 대림로 183,7호선 대림(구로구청),368.888504,NaN,대림,대림역 들꽃누리
34,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 압구정로 461,7호선 청담,675.120538,NaN,청담,청담역 커피빈
37,건대포토카페프린세스,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 동일로20길 98,7호선 건대입구,60.940672,NaN,건대입구,건대입구역 건대포토카페프린세스
47,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 삼성로 716,7호선 청담,229.848907,NaN,청담,청담역 커피빈
...,...,...,...,...,...,...,...,...,...
12149,안다미로팬케이크,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 군자로 19,7호선 어린이대공원(세종대),355.054760,NaN,어린이대공원,어린이대공원역 안다미로팬케이크
12150,초코라떼,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 아차산로36길 5,7호선 어린이대공원(세종대),592.981894,NaN,어린이대공원,어린이대공원역 초코라떼
12152,카페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 연동로11길 19,7호선 온수(성공회대입구),567.166627,NaN,온수,온수역 카페
12159,코끼리키즈까페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 개봉로3길 36,7호선 광명사거리,478.139096,NaN,광명사거리,광명사거리역 코끼리키즈까페


In [4]:
items = data["검색어"][195:]
items

1432            뚝섬유원지역 쥬씨차얌
1443          공릉역 빽다방서울과기대점
1445           어린이대공원역 자북커피
1447     마들역 공부인스터디카페노원마들센타
1449          총신대입구역 커피즐겨찾기
                ...        
12149      어린이대공원역 안다미로팬케이크
12150          어린이대공원역 초코라떼
12152                온수역 카페
12159        광명사거리역 코끼리키즈까페
12170            학동역 ON&OFF
Name: 검색어, Length: 1684, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/1684
식당 존재 x
진행상황 : 2/1684
식당 존재
https://place.map.kakao.com/27393509
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/1684
식당 존재
https://place.map.kakao.com/1040814371
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/1684
식당 존재
https://place.map.kakao.com/1733725595
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/1684
식당 존재 x
진행상황 : 6/1684
식당 존재 x
진행상황 : 7/1684
식당 존재
https://place.map.kakao.com/878570951
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 8/1684
식당 존재
https://place.map.kakao.com/1097099031
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 9/1684
식당 존재
https://place.map.kakao.com/2006325939
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/1684
식당 존재
https://place.map.kakao.com/1200660131
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 11/1684
식당 존재
https://place.map.kakao.com/1761578105
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/1684
식당 존재 x
진행상황 : 13/1684
식당 존재 x
진행상황 : 14/1684
식당 존재 x
진행상황 : 15/1684
식당 존재 x
진행상황 : 16/1684
식당 존재 x
진행상황 : 17/1684
식당 존재 x
진행상황 : 18/1684
식당 존재 x
진행상황 : 19/1684
식당 존재 x
진행상황 : 20/1684
식당 존재 x
진행상황 

식당 존재 x
진행상황 : 284/1684
식당 존재 x
진행상황 : 285/1684
식당 존재 x
진행상황 : 286/1684
식당 존재 x
진행상황 : 287/1684
식당 존재 x
진행상황 : 288/1684
식당 존재 x
진행상황 : 289/1684
식당 존재 x
진행상황 : 290/1684
식당 존재 x
진행상황 : 291/1684
식당 존재 x
진행상황 : 292/1684
식당 존재 x
진행상황 : 293/1684
식당 존재 x
진행상황 : 294/1684
식당 존재 x
진행상황 : 295/1684
식당 존재 x
진행상황 : 296/1684
식당 존재 x
진행상황 : 297/1684
식당 존재 x
진행상황 : 298/1684
식당 존재 x
진행상황 : 299/1684
식당 존재 x
진행상황 : 300/1684
식당 존재 x
진행상황 : 301/1684
식당 존재 x
진행상황 : 302/1684
식당 존재 x
진행상황 : 303/1684
식당 존재 x
진행상황 : 304/1684
식당 존재 x
진행상황 : 305/1684
식당 존재 x
진행상황 : 306/1684
식당 존재 x
진행상황 : 307/1684
식당 존재 x
진행상황 : 308/1684
식당 존재 x
진행상황 : 309/1684
식당 존재 x
진행상황 : 310/1684
식당 존재 x
진행상황 : 311/1684
식당 존재 x
진행상황 : 312/1684
식당 존재 x
진행상황 : 313/1684
식당 존재 x
진행상황 : 314/1684
식당 존재 x
진행상황 : 315/1684
식당 존재 x
진행상황 : 316/1684
식당 존재
https://place.map.kakao.com/818423534
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 317/1684
식당 존재
https://place.map.kakao.com/1795640660
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 318/1684
식당 존재
https://place.ma

3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 427/1684
식당 존재
https://place.map.kakao.com/528767186
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 428/1684
식당 존재
https://place.map.kakao.com/9449957
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 429/1684
식당 존재 x
진행상황 : 430/1684
식당 존재
https://place.map.kakao.com/630372352
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 431/1684
식당 존재
https://place.map.kakao.com/525622050
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 432/1684
식당 존재 x
진행상황 : 433/1684
식당 존재
https://place.map.kakao.com/564035782
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 434/1684
식당 존재
https://place.map.kakao.com/88749394
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 435/1684
식당 존재
https://place.map.kakao.com/1221715248
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 436/1684
식당 존재
https://place.map.kakao.com/27092771
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 437/1684
식당 존재
https://place.map.kakao.com/348013507
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 438/1684
식당 존재 x
진행상황 : 439/1684
식당 존재 x
진행상황 : 440/1684
식당 존재
https://place.map.kakao.com/1283573138
1
더 이상 리뷰 존재 X
기본

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 544/1684
식당 존재
https://place.map.kakao.com/519082124
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 545/1684
식당 존재 x
진행상황 : 546/1684
식당 존재
https://place.map.kakao.com/17007924
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 547/1684
식당 존재
https://place.map.kakao.com/8403571
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 548/1684
식당 존재
https://place.map.kakao.com/606933069
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 549/1684
식당 존재 x
진행상황 : 550/1684
식당 존재 x
진행상황 : 551/1684
식당 존재
https://place.map.kakao.com/2056453882
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 552/1684
식당 존재
https://place.map.kakao.com/901127235
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 553/1684
식당 존재
https://place.map.kakao.com/372548686
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 554/1684
식당 존재
https://place.map.kakao.com/1776547540
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 555/1684
식당 존재
https://place.map.kakao.com/537590852
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 556/1684
식당 존재 x
진행상황 : 557/1684
식당 존재
https://place.map.kakao.com/69565085

식당 존재
https://place.map.kakao.com/1656120674
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 659/1684
식당 존재
https://place.map.kakao.com/27588141
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 660/1684
식당 존재
https://place.map.kakao.com/578684213
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 661/1684
식당 존재
https://place.map.kakao.com/27442037
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 662/1684
식당 존재
https://place.map.kakao.com/17806448
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 663/1684
식당 존재
https://place.map.kakao.com/1533737371
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 664/1684
식당 존재
https://place.map.kakao.com/1506839020
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 665/1684
식당 존재
https://place.map.kakao.com/13725835
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 666/1684
식당 존재 x
진행상황 : 667/1684
식당 존재
https://place.map.kakao.com/145938811
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 668/1684
식당 존재
https://place.map.kakao.com/1302193543
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 669/1684
식당 존재 x
진행상황 : 670/1684
식당 존재
https://place.map.kakao.com/206731343
2

...

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1172/1684
식당 존재
https://place.map.kakao.com/1498885347
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1173/1684
식당 존재 x
진행상황 : 1174/1684
식당 존재
https://place.map.kakao.com/25693014
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1175/1684
식당 존재 x
진행상황 : 1176/1684
식당 존재 x
진행상황 : 1177/1684
식당 존재
https://place.map.kakao.com/23823579
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1178/1684
식당 존재 x
진행상황 : 1179/1684
식당 존재 x
진행상황 : 1180/1684
식당 존재 x
진행상황 : 1181/1684
식당 존재
https://place.map.kakao.com/24018517
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1182/1684
식당 존재
https://place.map.kakao.com/16587715
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1183/1684
식당 존재 x
진행상황 : 1184/1684
식당 존재
https://place.map.kakao.com/18612638
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1185/1684
식당 존재
https://place.map.kakao.com/21413240
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1186/1684
식당 존재
https://place.map.kakao.com/1213262105
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1187/1684
식당 존재
https://place.map.kakao.com/406977348
0
리뷰가 없습니다
더 이상 리뷰 

식당 존재
https://place.map.kakao.com/19485985
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1293/1684
식당 존재
https://place.map.kakao.com/706132623
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1294/1684
식당 존재
https://place.map.kakao.com/19596993
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1295/1684
식당 존재 x
진행상황 : 1296/1684
식당 존재 x
진행상황 : 1297/1684
식당 존재 x
진행상황 : 1298/1684
식당 존재 x
진행상황 : 1299/1684
식당 존재
https://place.map.kakao.com/1894613118
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1300/1684
식당 존재
https://place.map.kakao.com/944936465
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1301/1684
식당 존재
https://place.map.kakao.com/2098347092
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1302/1684
식당 존재 x
진행상황 : 1303/1684
식당 존재
https://place.map.kakao.com/27403006
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1304/1684
식당 존재
https://place.map.kakao.com/669545336
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1305/1684
식당 존재 x
진행상황 : 1306/1684
식당 존재 x
진행상황 : 1307/1684
식당 존재
https://place.map.kakao.com/931128282
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 :

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1414/1684
식당 존재
https://place.map.kakao.com/13494402
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1415/1684
식당 존재 x
진행상황 : 1416/1684
식당 존재 x
진행상황 : 1417/1684
식당 존재
https://place.map.kakao.com/1804703458
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1418/1684
식당 존재
https://place.map.kakao.com/22449673
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1419/1684
식당 존재
https://place.map.kakao.com/1989237181
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1420/1684
식당 존재 x
진행상황 : 1421/1684
식당 존재
https://place.map.kakao.com/11435932
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1422/1684
식당 존재 x
진행상황 : 1423/1684
식당 존재
https://place.map.kakao.com/10822361
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1424/1684
식당 존재
https://place.map.kakao.com/108515936
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1425/1684
식당 존재
https://place.map.kakao.com/1983959004
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1426/1684
식당 존재
https://place.map.kakao.com/892790496
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1427/1684
식당 존재
https://place.map.kakao.com/1337784803
5

1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1521/1684
식당 존재 x
진행상황 : 1522/1684
식당 존재
https://place.map.kakao.com/2099255771
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1523/1684
식당 존재
https://place.map.kakao.com/1167221894
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1524/1684
식당 존재
https://place.map.kakao.com/538398876
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1525/1684
식당 존재
https://place.map.kakao.com/1366601562
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1526/1684
식당 존재
https://place.map.kakao.com/1373628634
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1527/1684
식당 존재
https://place.map.kakao.com/489503293
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1528/1684
식당 존재
https://place.map.kakao.com/601004893
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1529/1684
식당 존재
https://place.map.kakao.com/1445847922
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1530/1684
식당 존재
https://place.map.kakao.com/26316695
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1531/1684
식당 존재
https://place.map.kakao.com/1050506421
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1532/1684

식당 존재
https://place.map.kakao.com/7962109
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1630/1684
식당 존재
https://place.map.kakao.com/29944633
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1631/1684
식당 존재
https://place.map.kakao.com/1981903487
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1632/1684
식당 존재 x
진행상황 : 1633/1684
식당 존재 x
진행상황 : 1634/1684
식당 존재
https://place.map.kakao.com/1412462373
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1635/1684
식당 존재 x
진행상황 : 1636/1684
식당 존재
https://place.map.kakao.com/25685187
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1637/1684
식당 존재
https://place.map.kakao.com/22674971
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1638/1684
식당 존재
https://place.map.kakao.com/27246680
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1639/1684
식당 존재 x
진행상황 : 1640/1684
식당 존재
https://place.map.kakao.com/782940805
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1641/1684
식당 존재 x
진행상황 : 1642/1684
식당 존재
https://place.map.kakao.com/351679360
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1643/1684
식당 존재
https://place.map.kakao.com/12526363
5
더 이상 리뷰 존재 X
기본

In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,d1sba1,공릉역 빽다방서울과기대점,1점,2020.11.22.
1,None,공릉역 빽다방서울과기대점,None,2017.09.08.
2,None,공릉역 빽다방서울과기대점,None,2017.06.20.
3,None,공릉역 빽다방서울과기대점,None,2017.03.19.
4,None,공릉역 빽다방서울과기대점,None,2016.12.11.
...,...,...,...,...
4355,15j6rj5,온수역 카페,5점,2020.05.15.
4356,17htnec,온수역 카페,5점,2020.04.07.
4357,11nt7ja,온수역 카페,5점,2020.01.21.
4358,redu68,온수역 카페,5점,2019.12.31.


In [8]:
img_elem

,ItemID,img_url
0,공릉역 빽다방서울과기대점,None
1,공릉역 빽다방서울과기대점,None
2,공릉역 빽다방서울과기대점,None
3,공릉역 빽다방서울과기대점,None
4,공릉역 빽다방서울과기대점,None
...,...,...
4355,온수역 카페,None
4356,온수역 카페,None
4357,온수역 카페,None
4358,온수역 카페,None


In [9]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
24,카페오작교,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 도산대로92길 26,7호선 청담,398.760647,NaN,청담,청담역 카페오작교
25,들꽃누리,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 대림로 183,7호선 대림(구로구청),368.888504,https://place.map.kakao.com/27393509,대림,대림역 들꽃누리
34,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 압구정로 461,7호선 청담,675.120538,https://place.map.kakao.com/1040814371,청담,청담역 커피빈
37,건대포토카페프린세스,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 동일로20길 98,7호선 건대입구,60.940672,https://place.map.kakao.com/1733725595,건대입구,건대입구역 건대포토카페프린세스
47,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 삼성로 716,7호선 청담,229.848907,None,청담,청담역 커피빈
...,...,...,...,...,...,...,...,...,...
12149,안다미로팬케이크,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 군자로 19,7호선 어린이대공원(세종대),355.054760,NaN,어린이대공원,어린이대공원역 안다미로팬케이크
12150,초코라떼,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 아차산로36길 5,7호선 어린이대공원(세종대),592.981894,NaN,어린이대공원,어린이대공원역 초코라떼
12152,카페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 연동로11길 19,7호선 온수(성공회대입구),567.166627,NaN,온수,온수역 카페
12159,코끼리키즈까페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 개봉로3길 36,7호선 광명사거리,478.139096,NaN,광명사거리,광명사거리역 코끼리키즈까페


In [10]:
img_elem

,ItemID,img_url
0,공릉역 빽다방서울과기대점,None
1,공릉역 빽다방서울과기대점,None
2,공릉역 빽다방서울과기대점,None
3,공릉역 빽다방서울과기대점,None
4,공릉역 빽다방서울과기대점,None
...,...,...
4355,온수역 카페,None
4356,온수역 카페,None
4357,온수역 카페,None
4358,온수역 카페,None


In [11]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,d1sba1,공릉역 빽다방서울과기대점,1점,2020.11.22.
1,None,공릉역 빽다방서울과기대점,None,2017.09.08.
2,None,공릉역 빽다방서울과기대점,None,2017.06.20.
3,None,공릉역 빽다방서울과기대점,None,2017.03.19.
4,None,공릉역 빽다방서울과기대점,None,2016.12.11.
...,...,...,...,...
4355,15j6rj5,온수역 카페,5점,2020.05.15.
4356,17htnec,온수역 카페,5점,2020.04.07.
4357,11nt7ja,온수역 카페,5점,2020.01.21.
4358,redu68,온수역 카페,5점,2019.12.31.


In [12]:
review_elem

,ItemID,review
0,공릉역 빽다방서울과기대점,여기 다른 지점보다 커피가 연해요
1,공릉역 빽다방서울과기대점,가격 굿
2,공릉역 빽다방서울과기대점,최고예요
3,공릉역 빽다방서울과기대점,최고예요
4,공릉역 빽다방서울과기대점,좋아요
...,...,...
4355,온수역 카페,건강한 메뉴도 많고 너무 맛있었어요
4356,온수역 카페,ㄱ
4357,온수역 카페,공간이 넓고 쾌적해요 사장님이 친절하세요
4358,온수역 카페,카페 자체가 깔끔하고 티&커피 메뉴도 다양. 베이커리도 종류는 많지않지만 수제티 팍...


In [13]:
img_elem.head()

,ItemID,img_url
0,공릉역 빽다방서울과기대점,None
1,공릉역 빽다방서울과기대점,None
2,공릉역 빽다방서울과기대점,None
3,공릉역 빽다방서울과기대점,None
4,공릉역 빽다방서울과기대점,None


In [14]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID  ItemID Rating    Timestamp
4353   7suuhu  온수역 카페     5점  2020.08.29.
4354  11muvbf  온수역 카페     5점  2020.06.22.
4355  15j6rj5  온수역 카페     5점  2020.05.15.
4356  17htnec  온수역 카페     5점  2020.04.07.
4357  11nt7ja  온수역 카페     5점  2020.01.21.
4358   redu68  온수역 카페     5점  2019.12.31.
4359   lrovug  온수역 카페     5점  2019.05.07.
      ItemID img_url
4353  온수역 카페    None
4354  온수역 카페    None
4355  온수역 카페    None
4356  온수역 카페    None
4357  온수역 카페    None
4358  온수역 카페    None
4359  온수역 카페    None
      ItemID                                             review
4353  온수역 카페  온수역 부근에 얼마 없는 고즈넉하고 넓직한 카페입니다. 커피,차 종류가 다양하고 통...
4354  온수역 카페  카페도 넓고 깨끗합니다  노래소리가 좀 크지만 역바로 앞이라 가까워서 좋네요  음료...
4355  온수역 카페                                건강한 메뉴도 많고 너무 맛있었어요
4356  온수역 카페                                                  ㄱ
4357  온수역 카페                             공간이 넓고 쾌적해요 사장님이 친절하세요
4358  온수역 카페  카페 자체가 깔끔하고 티&커피 메뉴도 다양. 베이커리도 종류는 많지않지만 수제티 팍...
4359  온수역 카페                 위치도 역 앞이라서 찾기 쉽고, 카

In [15]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_cafe(final).csv')

In [16]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_cafe_User(final).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_cafe_img(final).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline7_cafe_review(final).csv')

In [14]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


58

In [31]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_(1)263url.csv')